In [85]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from scipy.spatial.distance import braycurtis
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
import os
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from joblib import Parallel, delayed
from scipy.stats import gaussian_kde

In [86]:
import warnings
warnings.filterwarnings('ignore')

#### Choose your working directory

In [141]:
base_path=f"{os.getcwd()}/"

### Read train data

In [142]:
np.set_printoptions(suppress=True)

metadata_path = rf"{base_path}train_metadata.csv"
metadata_df = pd.read_csv(metadata_path)

data_path = rf"{base_path}train_data.csv"
data_df = pd.read_csv(data_path)

### Reading DFs for the final prediction
* We were given test data that contained both samaples with microbiome data and samples without it. Therefore, we merged the train data we were given at milestone 1 with the test samples that came with microbiome data (x_train and y_train).
* The actual test data is the test metadata that did not contain microbiome data (x_test). Predicted microbiome data will be submitted.

In [143]:
test_metadata_path = f"{base_path}test_metadata.csv"
test_metadata_df = pd.read_csv(test_metadata_path)
test_metadata_df.set_index('sample',inplace=True)

test_data_path = f"{base_path}short_timeseries_data.csv"
test_data_df = pd.read_csv(test_data_path)
test_data_df.set_index('sample',inplace=True)

# X_test
indexes_with_no_data = test_metadata_df.index.difference(test_data_df.index)
actual_test_metadata_df=test_metadata_df.loc[indexes_with_no_data,:]

# X_train
updated_train_metadata_df=pd.concat([metadata_df.set_index('sample'),test_metadata_df.loc[test_data_df.index,:]])
updated_train_metadata_df.reset_index(inplace=True)

# y_train
updated_train_data_df=pd.concat([data_df.set_index('sample'),test_data_df])
updated_train_data_df.reset_index(inplace=True)


### Preprocess

In [136]:
def preprocess(metadata_df, data_df, data_exists=True):

  if data_exists:
    data_columns=list(data_df.columns)
    metadata_columns=list(metadata_df.columns)

    # sample will be set as index
    data_columns.remove("sample")
    metadata_columns.remove("sample")

    # merge to keep all rows aligned in metadata and data 
    merged_df=data_df.merge(metadata_df,on='sample', how='inner')
    merged_df = merged_df.set_index('sample')

    # sort by date
    merged_df = merged_df.sort_values(by='collection_date')
    
    # split after sort, copy so the original will not be changed
    sorted_metadata_df=merged_df[metadata_columns].copy()
    sorted_data_df=merged_df[data_columns].copy()
  else:
    sorted_metadata_df=metadata_df.copy().sort_values(by='collection_date')
    sorted_data_df=None

  samples_for_index = sorted_metadata_df.index

  # create time features
  sorted_metadata_df[['year', 'month', 'day']] = sorted_metadata_df['collection_date'].str.split('-', expand=True).apply(pd.to_numeric) # split columns manually
  sorted_metadata_df['collection_date'] = pd.to_datetime(sorted_metadata_df['collection_date'])
  sorted_metadata_df['date_diff'] = sorted_metadata_df['collection_date'].diff().dt.days
  # fill NA: first value will be 0
  sorted_metadata_df['date_diff'] = sorted_metadata_df['date_diff'].fillna(0) 

  sorted_metadata_df.drop(columns=['collection_date'], inplace=True)

  # catagorial to numerical
  potenital_categorial = ['baboon_id', 'sex', 'social_group', 'season']
  categorical_columns = [col for col in potenital_categorial if col in sorted_metadata_df.columns] # try run with and without social group!
  encoder = OneHotEncoder(sparse_output=False)
  X_categorical = encoder.fit_transform(sorted_metadata_df[categorical_columns])
  X_categorical_df = pd.DataFrame(X_categorical, columns=encoder.get_feature_names_out(categorical_columns), index=samples_for_index)
  X_numerical_df = sorted_metadata_df[[col for col in sorted_metadata_df.columns if col not in categorical_columns]]

  sorted_metadata_df = pd.concat([X_categorical_df, X_numerical_df], axis=1)

  print(f'number of features after onehot: {sorted_metadata_df.shape[1]}')
  print(sorted_metadata_df.columns.tolist())

  return sorted_metadata_df , sorted_data_df


### Split Data
* Naive splitting the dataset into train and test, 20% test, 80% train. samples are sorted by data, so most recent samples are chosen to be test.
    * Was eventually not used.

In [113]:
def preprocess_after_filter(metadata_df, data_df):

  X=metadata_df.values
  y = data_df[data_df.columns].values
  split_index = int(0.8 * len(metadata_df))
  X_train, X_test = X[:split_index], X[split_index:]
  y_train, y_test = y[:split_index], y[split_index:]

  return X_train, X_test, y_train, y_test, data_df.columns


* Splitting the data by baboon ID, so the train will contain 80% of the baboons and the test will contain 20%. Allows us to evaluate our model on new set of Baboons that the model was not trained on.
    * Random split means the train dataset size vary, but we see it stays constant around +-4900.
    * Chosen for our model.

In [114]:
def split_per_monkey(metadata_df, data_df):
    
    # Get the unique IDs
    baboon_id_columns = [col for col in metadata_df.columns if col.startswith('baboon_id_Baboon_')]
    unique_baboon_ids = [col.split('_')[-1] for col in baboon_id_columns]
    
    # Split the baboon IDs into train and test sets (80% train, 20% test)
    train_baboons, test_baboons = train_test_split(unique_baboon_ids, test_size=0.2)
    
    print("number of train baboons", len(train_baboons))
    print("number of test baboons", len(test_baboons))
    
    # Create masks to filter metadata_df and data_df for training and testing based on baboon IDs
    train_mask = metadata_df[baboon_id_columns].apply(lambda row: any(row[col] == 1 for col in baboon_id_columns if col.split('_')[-1] in train_baboons), axis=1)
    test_mask = metadata_df[baboon_id_columns].apply(lambda row: any(row[col] == 1 for col in baboon_id_columns if col.split('_')[-1] in test_baboons), axis=1)
    
    x_train = metadata_df[train_mask]
    x_test = metadata_df[test_mask]
    y_train = data_df[train_mask]
    y_test = data_df[test_mask]

    print(f'number of samples in train dataset: {len(x_train)}')
    
    return x_train, x_test, y_train, y_test



### Add Features
* Add previous samples to the train metadata:
    * n_prevs: number of added samples
    * Use all microbe columns if microb == "", otherwise use the specified microbe column
    * deprecated but stays for future directions: runs PCA on those columns only if microb="" (does not predict each microbe seperetaly)

In [115]:
def add_prev(metadata_df, data_df, n_prevs, microb=""):

    combined_df = metadata_df.join(data_df)

    # Use all microbe columns if microb == "", otherwise use the specified microbe column
    if microb == "":
        microbe_columns = [col for col in data_df.columns if col.startswith('g_')]
    else:
        if isinstance(microb, list): # recieved more then one -> list
            microbe_columns=microb
        else:
            microbe_columns = [microb] # recieved one -> string

    if n_prevs == 0:
        return metadata_df

    for prev in range(1, n_prevs + 1):
        print("doing prev number", prev)
        # New column names for previous microbe data, init with NA
        previous_microbe_columns = [f'prev{prev}_{col}' for col in microbe_columns]
        combined_df[previous_microbe_columns] = None

        for baboon_id in [col for col in combined_df.columns if 'baboon_id' in col]:
            # get the df indexes in which the baboon appears
            baboon_indices = combined_df[combined_df[baboon_id] == 1].index.tolist()

            # init average value for early samples (no prev)
            avg_val=combined_df[combined_df[baboon_id] == 1][microbe_columns].mean(axis=0)
            current_index = [baboon_indices[i] for i in range(0,min(prev,len(baboon_indices)))]
            combined_df.loc[current_index, previous_microbe_columns] = avg_val.values
            
            # ADD PREV!
            for i in range(prev, len(baboon_indices)):
                current_index = baboon_indices[i]
                previous_index = baboon_indices[i - prev]
                combined_df.loc[current_index, previous_microbe_columns] = combined_df.loc[previous_index, microbe_columns].values

        # Fill NaN values with 0
        combined_df[previous_microbe_columns] = combined_df[previous_microbe_columns].fillna(0)

        # If using all microbiome (microb == ""), apply scaling and PCA
        if microb == "" and False:
            pca = PCA(n_components=2)
            normalized_data = normalize(combined_df[previous_microbe_columns], norm='l1', axis=1)
            pca_result = pca.fit_transform(normalized_data)


            # Create PCA column names for the reduced components
            pca_columns = [f'prev{prev}_PC1', f'prev{prev}_PC2']
            combined_df[pca_columns] = pca_result

            # Only keep the PCA columns in the final output
            result_df = combined_df[pca_columns]
        else:
            # keep the original previous microbe columns
            result_df = combined_df[previous_microbe_columns]

        # Merge the result back into metadata
        metadata_df = metadata_df.join(result_df)


    return metadata_df


### Create prediction
1. Process test metadata: add initial PREV generated by the train data distribution. Subsequential samples' PREV will be added iteritivly. 

In [116]:
# microbiome_lst - all microbiome. next func takes only the required data
def process_test_metadata(test_metadata_df, train_data_df, n_prevs):
    # no prev -> no need to add anything. return.
    if n_prevs==0:
        return test_metadata_df
    
    baboon_columns = [col for col in test_metadata_df.columns if col.startswith('baboon_id')]
    max_samples = int(test_metadata_df[baboon_columns].sum(axis=0).max()) # max number of samples per baboon
    dfs_list = []
    
    # create chunks of sub dataframes, where each df contains at most one row from each baboon.
    # data is sorted by date from preprocess func.
    # will be predicted in that order to ensure PREV already exists.
    for i in range(max_samples):
        ith_samples = pd.DataFrame()
        for baboon_col in baboon_columns:
            baboon_samples = test_metadata_df[test_metadata_df[baboon_col] == 1]
            if i < len(baboon_samples):  # Check if the ith sample exists for this baboon
                ith_samples = pd.concat([ith_samples, baboon_samples.iloc[[i]]])  # Append ith sample
        dfs_list.append(ith_samples)

    # use KDE to init test microbiome values from train distribution
    col_dists = {}
    microbiome_cols=train_data_df.columns
    for col in microbiome_cols:
        kde = gaussian_kde(train_data_df[col])  
        col_dists[col] = kde
    
    # fill missing columns with values from distribition
    for i in range(len(dfs_list)):
        chunk=dfs_list[i]
        for j in range(1, n_prevs + 1):
            for col in microbiome_cols:
                prev_col = f"prev{j}_{col}"
                if prev_col not in chunk.columns:
                    chunk[prev_col] = col_dists[col].resample(len(chunk))[0]  

    return dfs_list


2. Predict test: create a iterative prediction for test metadata (x_test), PREV are added in each iteration.
    * microbiome_lst: microbes that will be used as PREV features
    * microb: if given the function will only predict for it
    * selected_features: ones selected in the pipeline

In [117]:
def predict_test(microbiome_lst, model, test_metadata_df_arg, n_prevs, selected_features,dfs_lst_arg, microb=""):
    
    # assures no override
    test_metadata_df=test_metadata_df_arg.copy()

    # Use all microbe columns if microb == "", otherwise use the specified microbe column
    if microb == "":
        predict_columns = microbiome_lst
    else:
        if isinstance(microb, list): # recieved more then one -> list
            predict_columns=microb
        else:
            predict_columns = [microb] # recieved one -> string

    # no PREV needed: predict normally, no need in iterative prediction
    if n_prevs == 0:
        res_df = pd.DataFrame(model.predict(test_metadata_df[selected_features].to_numpy()), columns=predict_columns, index=test_metadata_df.index)
        return res_df
    
    # when runnign all hyperparameters in a loop, need a copy for each code execution so there are no overrides
    dfs_list=[df.copy() for df in dfs_lst_arg]

    all_preds_list = []
  
    print('start predict')
    for i in range(len(dfs_list)):
        chunk=dfs_list[i]
        
        # create a prediction and concat as df to create df with all predictions
        y_pred = model.predict(chunk[selected_features].to_numpy())
        y_pred_df = pd.DataFrame(y_pred, columns=predict_columns, index=chunk.index)
        all_preds_list.append(y_pred_df)

        # write current predicted values as PREV features in the next chuncks
        for j in range(1, n_prevs+1):
            if i+j < len(dfs_list):
                next_chunk=dfs_list[i+j] 
                next_chunk = next_chunk.merge(y_pred_df,how='left',on='sample',suffixes=["",f"_{j}_prev"])
                rename_cols = {f"{microbe}_{j}_prev": f"prev{j}_{microbe}" for microbe in microbiome_lst}
                next_chunk.rename(columns=rename_cols, inplace=True)


    y_pred_df_all = pd.concat(all_preds_list)
    print('created predictions')

    return y_pred_df_all

    
    

### Evaluate model
* Normalize results
* Use Bray Curtis to evaluate the prediction

In [118]:
def calc_score(y_pred, y_test, method_dic, is_microbe_wise):
    
    # Calculate the Bray-Curtis dissimilarity for each label column
    bray_curtis_dissimilarity = [i for i in range(len(y_pred))]
    for i in range(len(y_pred)):
        bray_curtis_dissimilarity[i] = braycurtis(y_test[i, :], y_pred[i, :])

    print('average bray curits dis is:', np.average(bray_curtis_dissimilarity))

    # save result to local dataframe
    if is_microbe_wise:
        path_to_csv=os.path.join(base_path,'microbe_wise_output.csv')
    else:
        path_to_csv=os.path.join(base_path,'whole_data_analysis_output.csv')

    if not os.path.exists(path_to_csv):
        df = pd.DataFrame(columns=['selection', 'rf_0_prevs', 'lgbm_0_prevs', 'rf_3_prevs', 'lgbm_3_prevs', 'rf_6_prevs', 'lgbm_6_prevs', 'rf_10_prevs', 'lgbm_10_prevs'])
        df.to_csv(path_to_csv, index=False)

    df=pd.read_csv(path_to_csv,header=0)
    df.set_index('selection', inplace=True)
    
    f_o_w=f"{method_dic['filter']}{method_dic['wrapper']}"
    if f_o_w=="": ind='No Selection'
    else: ind=f"{method_dic['filter']}{method_dic['wrapper']}_{method_dic['n_features']}"

    col=f"{method_dic['model']}_{method_dic['n_prevs']}_prevs"
    df.at[ind, col] = f"{np.average(bray_curtis_dissimilarity):.4f}"
    print(ind,col)

    display(df)
    df.to_csv(path_to_csv,index=True)

In [119]:
def normalize_result(y_matrix):
   y_normed = []
   for row in y_matrix:
     row_normed = row/sum(row)
     y_normed.append(row_normed)
   return np.array(y_normed)

In [120]:
def evaluate(y_test, y_predict, method, is_microbe_wise=False):
  # reorder them so we can compare microbiome with numpy, and not pandas
  y_predict = y_predict.reindex(y_test.index)

  # normalize the results- microbiome sum to 1
  y_predict_normalized = normalize_result(y_predict.values)

  # braycurtis etc.
  calc_score(y_predict_normalized, y_test.values, method, is_microbe_wise)
  print('\ncalculated score')

# Feature Selection

#### Correlation-based Selection: 
* Select features based on their correlation with the target variable (e.g., Pearson or Spearman correlation).
* can be chosen my average / medain of all values across the microbiome- we chose average

In [93]:
def correlation_feature_selection(processed_metadata_df, data_df, method, n_features):
  print(f'#### Method: {method} ####\n')
  microbiome_columns = data_df.columns

  # Iterate over each metadata feature and calculate correlation with each microbiome
  correlations = {}
  for meta_col in processed_metadata_df.columns:
      meta_correlations = []
      for cur_micro in microbiome_columns:
          corr_value = data_df[cur_micro].corr(processed_metadata_df[meta_col], method=method) #spearman / pearson
          meta_correlations.append(corr_value)

      correlations[meta_col] = meta_correlations

  # Convert to DataFrame with absolute values to infer significance
  correlation_df = pd.DataFrame.from_dict(correlations, orient='index',columns=microbiome_columns).abs()
  correlation_df['average']=correlation_df.mean(axis=1)
  correlation_df['median']=correlation_df.median(axis=1)

  n_chosen=correlation_df['average'].nlargest(n=n_features).index.values

  print('N chosen features:')
  print(n_chosen)

  return n_chosen



#### Variance Threshold: 
* Remove features with low variance (those that don’t change much across the dataset).
* we tested this feature with 3 different variance thresholds: 0.1 / 0.3 / 0.6

In [94]:
from sklearn.feature_selection import VarianceThreshold

In [65]:
def variance_feature_selection(processed_metadata_df, threshold=0.6):
  print(f'#### Threshold: {threshold} ####\n')

  processed_metadata_df_columns=processed_metadata_df.columns

  # create selector
  selector = VarianceThreshold(threshold=threshold)
  transformed_data = selector.fit_transform(processed_metadata_df[processed_metadata_df_columns])

  # Convert to DataFrame features > TH
  selected_features_df = pd.DataFrame(transformed_data, columns=[col for col in processed_metadata_df_columns if selector.variances_[processed_metadata_df_columns.index(col)] > threshold])

  print("Selected features based on variance threshold:\n")
  print(selected_features_df.columns)

  print(f"selected features count: {len(selected_features_df.columns)}")

  return list(selected_features_df.columns)




#### MinimumRedundancyMaximumRelevance: 
* a feature selection technique used to select a subset of features that are both highly relevant to the target variable and minimally redundant with each other.

* mrmr_classif, for feature selection when the target variable is categorical (binary or multiclass).
* mrmr_regression, for feature selection when the target variable is numeric.
* Note: the output of mrmr_classif is a list containing K selected features. This is a ranking, therefore, if you want to make a further selection, take the first elements of this list.

In [66]:
# source: https://github.com/smazzanti/mrmr
# !pip install mrmr_selection

In [41]:
import mrmr
from mrmr import mrmr_classif

In [ ]:
def fs_mrmr(processed_metadata_df,processed_data_df,n_features):

  d={k: 0 for k in processed_metadata_df.columns}

  # run uppon each microbe, the dict represents the number of times each feature was found most significant.
  # low value indicates low significance
  for micro in processed_data_df.columns:
    # return a list when lower index is more significant
    selected_features = mrmr_classif(X=processed_metadata_df, y=processed_data_df[micro], K=len(processed_metadata_df.columns))
    sig=0
    for f in selected_features:
      d[f]+=sig
      sig+=1
    print("mrmr for:", micro, selected_features)
  print(d)

  #take the n features with lowest values
  top_features = [key for key, value in sorted(d.items(), key=lambda item: item[1], reverse=False)[:n_features]] 

  return top_features


#### Recursive Feature Elimination (RFE): 
* Iteratively builds a model and removes the least important features.

In [67]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import Ridge

In [68]:
# Not parallel
def fs_RFE(X_train, y_train,n_features,model):

  # Initialize array to keep track of how many times each feature is selected
  best_features =np.array([0 for i in range(X_train.shape[1])])

  #for microbe in label_cols:
  for col in y_train.T:
    y_micro = col
    rfe = RFE(model, n_features_to_select=n_features)
    rfe.fit(X_train, y_micro) # y must be 1-D vector
    # best features: rfe.support_
    # returns a Boolean mask (array) of shape (n_features,), where True/False indicates if the feature was selected.

    # best features for this specific micro
    best_features = best_features + np.array(rfe.support_).astype(int)

  # find the top features in average - array of indexes - DEBATEABLE
  top_n_features = np.argsort(best_features)[-n_features:][::-1]


  return top_n_features

In [138]:
# with parallel n_jobs
def fs_RFE(X_train, y_train, n_features, model):

    n_jobs = 5

    # Initialize array to keep track of how many times each feature is selected
    best_features = np.zeros(X_train.shape[1], dtype=int)
    print("initialized best_features array, start proccessing columns")

    # runs in parallel for each output label
    def process_single_column(y_col):
        rfe = RFE(model, n_features_to_select=n_features)
        rfe.fit(X_train, y_col)  # Fit RFE for the given column
        print("at RFE, done with one")
        return np.array(rfe.support_).astype(int)

    # Parallel execution for each column in y_train.T
    results = Parallel(n_jobs=n_jobs, backend="threading")(delayed(process_single_column)(col) for col in y_train.T)

    for result in results:
        best_features += result

    # Find the top n features by average selection
    top_n_features = np.argsort(best_features)[-n_features:][::-1]

    return top_n_features


#### Forward Selection: 
* Starts with no features and adds one feature at a time based on model performance.
#### Backward Elimination: 
* Starts with all features and removes the least significant ones one by one.

In [129]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [130]:
is_forawrd = True # False for backwords

In [131]:
def fs_SFS(X_train,y_train,n_features, is_forawrd, model):

  sfs = SFS(model, k_features=n_features, forward=is_forawrd, floating=False)
  sfs.fit(X_train, y_train) # y is matrix
  selected_features_indices = list(sfs.k_feature_idx_)

  return selected_features_indices

#### Boruta algorithm: 
* built around the Random Forest classifier and aims to find features that are strongly or weakly relevant to the target variable.

In [61]:
# !pip install Boruta

In [24]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

import concurrent.futures


In [60]:
# Not parallel
### expects discrete values
def fs_boruta(X_train,y_train,n_features, model):

  best_features =np.array([0 for i in range(X_train.shape[1])])

  for col in y_train.T:
    y_micro = col
    boruta_selector = BorutaPy(model, n_estimators='auto', random_state=42)
    boruta_selector.fit(X_train, y_micro)

    # boruta_selector.support_ returns a Boolean mask (array) of shape (n_features,), where True/False indicates if the feature was selected.
    # best features for this specific microbe
    best_features = best_features + np.array(boruta_selector.support_).astype(int)

  # find the top features in average - array of indexes
  top_n_features = np.argsort(best_features)[-n_features:][::-1]


  return top_n_features

In [26]:
# Parallel processing

def process_boruta_for_column(X_train, y_micro, model):
    boruta_selector = BorutaPy(model, n_estimators='auto', random_state=42)
    boruta_selector.fit(X_train, y_micro)
    return np.array(boruta_selector.support_).astype(int)

def fs_boruta(X_train, y_train, n_features, model, label_cols):
    best_features = np.zeros(X_train.shape[1], dtype=int)

    # Use ProcessPoolExecutor for parallel processing
    with concurrent.futures.ProcessPoolExecutor(max_workers=20) as executor:
        # Submit jobs for each column in y_train
        futures = [executor.submit(process_boruta_for_column, X_train, col, model) for col in y_train.T]

        for future in concurrent.futures.as_completed(futures):
            best_features += future.result()

    # Find the top features in average - array of indexes
    top_n_features = np.argsort(best_features)[-n_features:][::-1]

    return top_n_features

### The FS Pipeline

In [46]:
def pipeline(processed_metadata_df, processed_data_df,filt,wrapper,n_features,is_forawrd,model_type, is_per_microbe):
  
  print("start pipeline")

  # init selected features to be all
  selected_features=np.array(processed_metadata_df.columns)

  # filter features before run model
  if filt:
    print(f'start filtering using: {filt}')
    if filt == 'corr':
      selected_features= correlation_feature_selection(processed_metadata_df, processed_data_df, 'pearson',n_features) #spearman
    if filt == 'var':
      selected_features=variance_feature_selection(processed_metadata_df)
    if filt == 'mrmr':
      selected_features=fs_mrmr(processed_metadata_df,processed_data_df,n_features)

    print('done filtering')
    processed_metadata_df = processed_metadata_df[list(selected_features)]


  print('start create model')
  x_train=processed_metadata_df.to_numpy()
  y_train=processed_data_df.to_numpy()

  if model_type == 'rf':
    model = RandomForestRegressor(n_jobs=3)
  if model_type == 'lgbm':
    base_model = lgb.LGBMRegressor(verbosity=-1)
    # Wrap the LightGBM regressor in MultiOutputRegressor to handle multi-output labels
    model = base_model
    if not is_per_microbe:
      model = MultiOutputRegressor(base_model)

  print('created model')

  if wrapper:
    print(f'start filtering using: {wrapper}')
    # fit wrapper model inside the function
    if wrapper == 'RFE':
      top_n_features=fs_RFE(x_train,y_train,n_features,model)
    if wrapper == 'SFS':
      top_n_features=fs_SFS(x_train,y_train,n_features, is_forawrd, model)
    if wrapper == 'Boruta':
      top_n_features=fs_boruta(x_train,y_train,n_features, model)
    
    features = processed_metadata_df.columns
    selected_features = features[top_n_features] 

    print(f'selected_features: {selected_features}')


    # exclude unselected features from x_train
    tmp_x=x_train[:, top_n_features]
    x_train=tmp_x

  print('start fit model')
  model.fit(x_train, y_train)
  print('done fit model')

  return model, selected_features


## RUN MODEL
* Option 1: Whole data version- predicting for the whole microbiome together
* Option 2: Predict for each microbe seperatly - create prediction for each microbe and then evaluate them all together
    * Unused due to long infeasible execution time

In [69]:
filt= None # None /'corr' / 'var' / 'mrmr'
wrapper= 'RFE' # None / RFE / SFS / Boruta
model_type = 'rf' # rf / knn / lgbm
n_features=80 #hyper parameter
is_forawrd=True
n_prevs=0
per_monkey = True

method={'filter':filt, 'wrapper':wrapper,'model':model_type,'n_features':n_features,'is_forawrd':is_forawrd,'n_prevs':n_prevs,'per_monkey':per_monkey}
method = {k: (v if v is not None else "") for k, v in method.items()}

### Whole data version

In [ ]:
def predict_for_all_microbe_together(metadata_df, data_df, test_data_df_arg=[]):

    processed_metadata_df, processed_data_df= preprocess(metadata_df, data_df)

    if len(test_data_df_arg)!=0:
        # if we want to predict for the actual test data- we need to split it accordingly
        x_train=processed_metadata_df
        y_train=processed_data_df
        x_test, y_test=preprocess(test_data_df_arg[0],None,False)
        print(f'number of samples in train dataset: {len(x_train)}')
        
    else:
        x_train, x_test, y_train, y_test = split_per_monkey(processed_metadata_df, processed_data_df)
    
    x_train = add_prev(x_train, y_train,n_prevs)

    # send only columns that do not contain our baboons- different in tarin and test
    model, selected_features=pipeline(x_train, y_train, filt, wrapper,n_features,is_forawrd,model_type, False)

    # some features only exist in train and not test -> need to find them and manualy add those columns to test
    test_baboons=[col for col in x_test.columns if 'baboon_id' in col]
    selected_features_unsafe=[col for col in selected_features.tolist() if ('baboon_id' in col and col not in test_baboons)]

    # fill 0 all required babbon_id in test features
    for i in range(len(selected_features)):
        if selected_features[i] in selected_features_unsafe:
            print(f'{selected_features[i]} not in test - fill 0')
            x_test[selected_features[i]]=0

    chunks=process_test_metadata(x_test, y_train, n_prevs)
            
    y_predict=predict_test(y_train.columns, model, x_test, n_prevs, selected_features, chunks)

    
    if len(test_data_df_arg)!=0:
        y_predict.to_csv(f'{base_path}whole_data_final_prediction.tsv', index=True,sep='\t')
        return
    
    evaluate(y_test, y_predict, method, is_microbe_wise=False)


# Only train
predict_for_all_microbe_together(metadata_df, data_df)

# Test for real predictions
predict_for_all_microbe_together(updated_train_metadata_df, updated_train_data_df, [actual_test_metadata_df])

### Predict for each microbe seperatly - \*unused\*

In [ ]:
# Get correlation between microbes
# choose pearson / spearman
def get_correlation_mat(data_df, method):
  print(f'#### Method: {method} ####\n')

  microbiome_columns = data_df.columns

  # Iterate over each microbe and calculate correlation with all microbiome
  correlations = pd.DataFrame(0, index=microbiome_columns, columns=microbiome_columns)

  for col in range(len(microbiome_columns)):
    for index in range(col,len(microbiome_columns)):
      corr=data_df[microbiome_columns[index]].corr(data_df[microbiome_columns[col]], method=method) #spearman pearson
      # Convert to absolute values to infer correlations between microbiome
      correlations.loc[microbiome_columns[index], microbiome_columns[col]] = abs(corr)
  
  correlations = correlations.where(np.tril(np.ones(correlations.shape)).astype(bool), correlations.T)
  
  print('correlations: \n')
  display(correlations)

  return correlations


In [ ]:
def predict_for_each_microbe_sep(metadata_df, data_df, test_data_df_arg=[]):

  processed_metadata_df, processed_data_df= preprocess(metadata_df, data_df)

  if len(test_data_df_arg)!=0:
        # if we want to predict for the actual test data- we need to split it accordingly
        x_train=processed_metadata_df
        y_train=processed_data_df
        x_test, y_test=preprocess(test_data_df_arg[0],None,False)
        print(x_test)
        print(f'number of samples in train dataset: {len(x_train)}')
  else:
    x_train, x_test, y_train, y_test = split_per_monkey(processed_metadata_df, processed_data_df)

  # add all prevs in one function call and then filter columns each time
  x_columns=x_train.columns
  x_train_prev = add_prev(x_train, y_train,n_prevs)

  # get correlations between microbiome
  corr_mat=get_correlation_mat(y_train, 'spearman')

  all_pred_df=pd.DataFrame()
  
  chunks=process_test_metadata(x_test, y_train, n_prevs)


  count = 0
  for microb in y_train.columns:
    print("calcing for", microb)

    # find the most correlated microbiome to add as features
    corr_microbiome=corr_mat[corr_mat[microb] >= 0.6].index.tolist() # can be hyper parameter , lower th will lead to a lot of correlated microbs
    print(f'found high correlations between: {corr_microbiome}')

    cur_x_columns=x_columns.to_list()+[f'prev{prev}_{microbe}' for microbe in corr_microbiome for prev in range(1, n_prevs+1)]
    x_train_microbe=x_train_prev[cur_x_columns]

    count +=1
    print(f'count: {count}')

    cur_y_train=y_train[[microb]]
    model, selected_features=pipeline(x_train_microbe,cur_y_train,filt,wrapper,n_features,is_forawrd,model_type, True)

    # some features only exist in train and not test -> need to find them and manualy add those columns to test
    test_baboons=[col for col in x_test.columns if 'baboon_id' in col]
    selected_features_unsafe=[col for col in selected_features.tolist() if ('baboon_id' in col and col not in test_baboons)]

    # fill 0 all required babbon_id in test features
    for i in range(len(selected_features)):
        if selected_features[i] in selected_features_unsafe:
            print(f'{selected_features[i]} not in test - fill 0')
            x_test[selected_features[i]]=0

    y_predict=predict_test(corr_microbiome, model,x_test, n_prevs, selected_features,chunks, microb)

    all_pred_df[microb]=y_predict # add microb prediction to df as new column


  if len(test_data_df_arg)!=0:
        all_pred_df.to_csv(f'{base_path}each_microbe_prediction.tsv', index=True,sep='\t')
        return
  
  evaluate(y_test, all_pred_df, method, is_microbe_wise=True)



# Only train
predict_for_each_microbe_sep(metadata_df, data_df)

# Test for real predictions
predict_for_each_microbe_sep(updated_train_metadata_df, updated_train_data_df, [actual_test_metadata_df])


## Graphs!

#### Heatmap

In [ ]:
def heatmap(is_microbe_wise,n_prev, show_none_only, n_features):
    if is_microbe_wise:
        path=os.path.join(base_path,'microbe_wise_output.csv')
        title='Microbe Wise'
    else:
        path=os.path.join(base_path,'whole_data_analysis_output.csv')
        title='Whole Data Analysis'
        
    df=pd.read_csv(path,header=0)
    df.set_index('selection', inplace=True)

    if n_prev:
        df=df[[col for col in df.columns if str(n_prev) in col]]
    if show_none_only:
        df=df.iloc[0:1,:]
    if n_features:
        df=df.T[[col for col in df.T.columns if str(n_features) in col]].T

    plt.figure(figsize=(5, 4))

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(df, annot=True, cmap='coolwarm', cbar_kws={'label': 'Value'}, fmt='.2f')

    plt.title(f'Heatmap - {title}')
    plt.xlabel('Model Type')
    plt.ylabel('Filter / Wrapper _ Number of Features')

    plt.show()


heatmap(False,False, False, False)